In [3]:
******* Exercise - 4**************

# what i have to do
# 1. we did f.one_hot which just makes input of 1 * (27*27) i.e 0 0 0 0 .... 1..0000 .
# 2. seems time_consuming to do this for every training example as we can just select the corresponding row in W.

# Ok let's do this. 
# it's simple just do logits = W[X_train].  (simple matrix multiplication)

In [49]:
# W = torch.rand(10,3)
# X_train = torch.tensor([2,5,8,1])
# xenc = F.one_hot(X_train, num_classes = 10).float()
# print(W[X_train])
# print(xenc@W)

In [50]:
# Yes they are same


In [51]:


import torch
words = open('names.txt','r').read().splitlines()

In [52]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [53]:
import torch.nn.functional as F

In [63]:
# create the dataset

xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append((ix1*27)+ix2)
    ys.append(ix3)
    
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27*27, 27), generator=g, requires_grad=True)

number of examples:  196113


In [64]:
from sklearn.model_selection import train_test_split

X_train,X_temp,y_train,y_temp = train_test_split(xs,ys,test_size = 0.2, random_state=42)
X_dev, X_test, y_dev, y_test = train_test_split(X_temp,y_temp, test_size=0.5, random_state = 42)

# X_train, y_train ma 80% data xa ani
#X_dev ra X_test ma 10 10% data xa ni

In [65]:
# print(X_train.size())
# print(X_dev.size())
# print(X_test.size())
train_size = len(X_train)
dev_size = len(X_dev)
test_size = len(y_test)
# X_train = X_train.float()
# y_train = y_train.float()
# X_dev = X_dev.float()
# y_dev = y_dev.float()
# y_train = y_train.float()
# y_test = y_test.float()

In [66]:
# NOW i have to create a variable to store the dev_loss every time. if at some point dev_loss(new) > dev_loss(past) 
#stop the training

#Also let's find after how many iteration we did early stopping
#Also let's see if doing early stopping make our results better

dev_loss = 1e10

In [67]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [70]:
for k in range(50):
    
    
    #forward pass for train-set
    
#     xenc = F.one_hot(X_train, num_classes = (27*27)).float() #input to the network: one-hot encoding
#     logits = no_of_examples * 27
    
    logits = W[X_train]  #log-counts predict gareko hai
    counts = logits.exp()  #coutns, equivalent to N
    probs = counts/counts.sum(1,keepdim = True) #probabilities for next character
    
    loss = -probs[torch.arange(train_size),y_train].log().mean() + 0.01 * (W**2).mean()  # i haven't understood this second part
    
    
    #forward pass for dev-set
    logits_dev = W[X_dev]
    counts_dev = logits_dev.exp()
    probs_dev = counts_dev/ counts_dev.sum(1, keepdim = True)
    loss_2 = -probs_dev[torch.arange(dev_size),y_dev].log().mean()
    
    print(f"dev_loss: {loss_2.item()}")
    
    if(dev_loss < loss_2):
        print(f"early stopping done, stoped at: {k}")
        break
    else:
        dev_loss = loss_2
    #maile bujeko yo loss chai,
#     probs[torch.arrange(num),ys] this gives probability predcited by our model for ground truth. if it is less than 
#     100% model we give it a loss i.e -loglikelihood now model updates it's parameters.
    
#     Q. I am super-amazed how this loss worked? 
#     --> because we are not always sure what letters comes after pairs in real names. any letter can come. 
#         but some are more likely.
#     --> i think our model will twiks our W such that it gives more probabilty for more likely words based on training set
    
#     Q. Still confused how loss will converged? I suspect it will not. (noob me, think later)
#     Q. 

     
    print(f"loss: {loss.item()}")
    #backward pass:
    
    W.grad = None # paile sabaile gradient lai zero gareko
    loss.backward()  #magic hehe
    
    #update
    
    W.data += -50*W.grad   #remember to increment opp to direction of gradient, otherwise face the consequences)

    

dev_loss: 2.3962433338165283
loss: 2.4097914695739746
dev_loss: 2.393950939178467
loss: 2.4074549674987793
dev_loss: 2.3916923999786377
loss: 2.4051523208618164
dev_loss: 2.389467239379883
loss: 2.402883291244507
dev_loss: 2.3872740268707275
loss: 2.400646924972534
dev_loss: 2.385112762451172
loss: 2.398442268371582
dev_loss: 2.3829824924468994
loss: 2.3962695598602295
dev_loss: 2.380882501602173
loss: 2.394127130508423
dev_loss: 2.378812074661255
loss: 2.392014503479004
dev_loss: 2.3767707347869873
loss: 2.3899309635162354
dev_loss: 2.374757766723633
loss: 2.387876510620117
dev_loss: 2.372772455215454
loss: 2.385850667953491
dev_loss: 2.370814800262451
loss: 2.3838515281677246
dev_loss: 2.3688836097717285
loss: 2.381880044937134
dev_loss: 2.366978645324707
loss: 2.379934310913086
dev_loss: 2.3650989532470703
loss: 2.3780152797698975
dev_loss: 2.3632445335388184
loss: 2.3761210441589355
dev_loss: 2.361414909362793
loss: 2.3742518424987793
dev_loss: 2.3596091270446777
loss: 2.3724076747

In [71]:
# i noticed if i change 0.01 to 0.0001 network learns slower
# ok this is not the case i think, does this have any advantages?

# if multiplicating factor(K)is too big, faster overfitting which helps to generalize the data?
#  if we decrease K, penalty term becomes less significant. so model may overfitt. (meaning good on train, bad on test)
#  what about small k with early stopping?

# ---> because we are using early stopping to prevent model from overfitting and using small k to get high accuracy?

# is this right?

# what i don't know
# Q.   What will happen if k tends to 0? (this means we don't use regularization)
# Q.   Why we need regularixation if we have early stopping?                                        

In [72]:


xenc_test = F.one_hot(X_test, num_classes = (27*27)).float()
logits_test = xenc_test @ W
counts_test = logits_test.exp()
probs_test = counts_test/ counts_test.sum(1, keepdim = True)
loss_3 = -probs_test[torch.arange(test_size),y_test].log().mean()

print(loss_3.item())

2.3275363445281982


In [85]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(10):

  out = []
  ix = 0
  gt = torch.randint(1,27,(1,)).item()  
  while True:

    # ----------
    # BEFORE:
    #p = P[ix]
    # ----------
    # NOW:
    ix = (ix*27) + (gt)
    out.append(itos[gt])
    xenc = F.one_hot(torch.tensor([ix]), num_classes=27*27).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------

    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    ix , gt = gt, ix
    
    if gt == 0:
      break
  print(''.join(out))

ta
jojded
olkcgaz
ma
josay
wein
sa
yuiir
gtohcassagezvksiania
der


In [ ]:
# # EX-3
# # What i have to do is:
# #     1. use dev-set to tune the strength of smoothing. 
# #     2. is this mean i have to find the best equation of the regularization to minimize the loss?
# #     3. what patterns i see in train and dev set loss as you tune this strength
    
    
# # My plans:
    
# #     1. First I will see what minimum loss i can get with early stopping
# #     2. I i will try different regularization to decrease the loss even further
    


# # conclusion: I got loss of 2.1 with small k and early stopping technique

# let's learn regularization.


# #weights haru lai zero ko close ma laaune is one form of label  smoothing, kinaki. if w sabai zero vayo vane.
# hamro logits ni zero hunxa. counts = 1 hunxa sabai ko. now probs ta sabai ko same hune vayo ni ta. 

# so smoothing garne ho vane, sabai zero ko najik ma aaye ramro?
# or (sabai W equal vayeni hola ni? hunna jasto lagyo. yes hune raixa copy ma hereko)

# what is difference between weights zero banaune and weights equal banaune?


 

#     + (W**2).sum() yo loss ma thapesi ta aba hamro model le parameters haru zero ko najik ma rakhne vayo ni ta.
    
#     (W**2) garepaxi negative ni positive vayo hai.
    
#     let's take mean sum ali large huna sakxa
    
#     + k* (W**2).mean()
    
#     yo multiplication factor (regularization strength) chai jati tholu vayo teti we punish for W not being close to zero. so ekdam close to zero
#     aauxa hola k tholo vayo vane
    
    
#     yo k kati rakhne? experiment garne??
    
    
# wait label smoothing kina garne ra????
# hamle +1 gareko theyo ni inf huna nadina, yesma jati thulo + garyo teti label smoothing hunxa. 

# Jati thulo + garyo is simlar to increasing k.   (wow just wow)



# Ok why do we did it?
# ->hamro data perfectly accurate ta hunna ni ta, aba mathi data ma j paxi q aaune zero theyo, tara real life ma ta testo hudaina
#   kunai na kunai ta hola j paxi q vako. so we add some number to consider those cases.
    
# Ok i found it just like bias.
# Is label smoothing somewhat similar to adding a bias??





# ok learned new things 

# we got 
# loss = -probs[torch.arrange(num), ys].log().mean() + 0.01*(W**2).mean() + (i can add term here too, to experiment and make my model better)
#                                                                           (if kei vaye loss decrease garna ni payo, or increase garna ni payo)
# nice.